In [ ]:
import os
import glob
import pandas as pd
import cv2
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

def contar_frames_folder(rgb_folder):
    """Cuenta el número de frames en una carpeta RGB"""
    try:
        # Contar archivos de imagen en la carpeta RGB
        frame_files = glob.glob(os.path.join(rgb_folder, "*.jpg")) + glob.glob(os.path.join(rgb_folder, "*.png"))
        return len(frame_files)
    except Exception as e:
        print(f"Error contando frames en {rgb_folder}: {e}")
        return 0

def contar_frames_video(video_path):
    """Cuenta el número de frames en un video"""
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return 0
        
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        return frame_count
    except Exception as e:
        print(f"Error contando frames en {video_path}: {e}")
        return 0

def contar_keyframes(keyframes_folder):
    """Cuenta el número de keyframes en una carpeta"""
    try:
        # Contar archivos de imagen en la carpeta keyframes
        keyframes_count = len(glob.glob(os.path.join(keyframes_folder, "*.jpg"))) + len(glob.glob(os.path.join(keyframes_folder, "*.png")))
        return keyframes_count
    except Exception as e:
        print(f"Error contando keyframes en {keyframes_folder}: {e}")
        return 0

def encontrar_videos_y_keyframes(ruta_base):
    """
    Recorre la estructura de carpetas para encontrar videos, frames y keyframes
    
    Returns:
        DataFrame con la información recopilada
    """
    datos = []
    
    # Buscar carpetas de sujetos
    sujeto_pattern = os.path.join(ruta_base, "SUJETO *")
    sujeto_folders = glob.glob(sujeto_pattern)
    
    print(f"Encontrados {len(sujeto_folders)} sujetos")
    
    for sujeto_folder in sorted(sujeto_folders):
        sujeto_name = os.path.basename(sujeto_folder)
        print(f"Procesando {sujeto_name}...")
        
        # Buscar carpeta de señas procesadas
        senas_folder = os.path.join(sujeto_folder, "señas_procesadas")
        if not os.path.exists(senas_folder):
            senas_folder = os.path.join(sujeto_folder, "senas_procesadas")
            
        if not os.path.exists(senas_folder):
            print(f"  No se encontró carpeta de señas procesadas para {sujeto_name}")
            continue
        
        # Buscar carpetas de expresiones
        expression_folders = [f for f in glob.glob(os.path.join(senas_folder, "*")) if os.path.isdir(f)]
        
        for expression_folder in sorted(expression_folders):
            expression_name = os.path.basename(expression_folder)
            print(f"  Procesando expresión: {expression_name}")
            
            # Buscar carpetas de muestras
            muestra_folders = [f for f in glob.glob(os.path.join(expression_folder, "muestra_*")) if os.path.isdir(f)]
            
            for muestra_folder in sorted(muestra_folders):
                muestra_name = os.path.basename(muestra_folder)
                print(f"    Procesando {muestra_name}")
                
                # IMPORTANTE: Buscar carpeta RGB para los frames originales
                rgb_folder = os.path.join(muestra_folder, "RGB")
                if not os.path.exists(rgb_folder):
                    rgb_folder = os.path.join(muestra_folder, "rgb")  # Intentar con minúsculas
                
                # Buscar carpeta keyframes
                keyframes_folder = os.path.join(muestra_folder, "keyframes") #AQUI DECLARO EL NOMBRE DE LA CARPETA QUE QUIERO BUSCAR
                
                # Buscar videos para obtener la ruta
                video_files = glob.glob(os.path.join(muestra_folder, "*.mp4"))
                
                # Si no se encuentran videos, intentar con nombres predecibles
                if not video_files:
                    print("    No se encontraron videos directamente. Intentando patrones de nombres comunes...")
                    
                    possible_names = [
                        f"{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                    ]
                    
                    for name in possible_names:
                        candidate = os.path.join(muestra_folder, name)
                        if os.path.exists(candidate):
                            video_files = [candidate]
                            print(f"    Encontrado video con patrón: {name}")
                            break
                
                # Obtener ruta del video (si existe)
                video_path = video_files[0] if video_files else ""
                
                # Contar frames originales
                frames_count = 0
                if os.path.exists(rgb_folder) and os.path.isdir(rgb_folder):
                    # Contar frames en la carpeta RGB
                    frames_count = contar_frames_folder(rgb_folder)
                    print(f"      Encontrados {frames_count} frames en carpeta RGB")
                elif video_path:
                    # Si no hay carpeta RGB pero hay video, contar frames del video
                    frames_count = contar_frames_video(video_path)
                    print(f"      Encontrados {frames_count} frames en el video")
                
                # Contar keyframes
                keyframes_count = 0
                if os.path.exists(keyframes_folder) and os.path.isdir(keyframes_folder):
                    keyframes_count = contar_keyframes(keyframes_folder)
                    print(f"      Encontrados {keyframes_count} keyframes")
                
                # Agregar datos al DataFrame solo si hay frames o keyframes
                if frames_count > 0 or keyframes_count > 0:
                    datos.append({
                        'Sujeto': sujeto_name,
                        'Categoría': expression_name,
                        'Muestra': muestra_name,
                        'Ruta_Video': video_path,
                        'Ruta_RGB': rgb_folder if os.path.exists(rgb_folder) else "",
                        'Ruta_Keyframes': keyframes_folder if os.path.exists(keyframes_folder) else "",
                        'Frames_Original': frames_count,
                        'Keyframes': keyframes_count
                    })
    
    # Crear DataFrame con todos los datos
    if datos:
        df = pd.DataFrame(datos)
        return df
    else:
        print("No se encontraron datos")
        return None

def crear_excel_con_hojas_por_sujeto(df, ruta_excel):
    """
    Crea un archivo Excel con una hoja por sujeto y una hoja de resumen
    
    Args:
        df: DataFrame con los datos
        ruta_excel: Ruta donde guardar el archivo Excel
    """
    # Crear un nuevo libro de Excel usando openpyxl
    wb = Workbook()
    
    # Eliminar la hoja predeterminada
    default_sheet = wb.active
    wb.remove(default_sheet)
    
    # Estilos
    header_font = Font(bold=True, size=12)
    header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    total_font = Font(bold=True)
    total_fill = PatternFill(start_color="E6E6E6", end_color="E6E6E6", fill_type="solid")
    border = Border(
        left=Side(style='thin'), 
        right=Side(style='thin'), 
        top=Side(style='thin'), 
        bottom=Side(style='thin')
    )
    
    # Diccionario para estadísticas de resumen
    resumen_stats = {}
    
    # Agrupar datos por sujeto
    sujetos = df['Sujeto'].unique()
    
    # Crear una hoja para cada sujeto
    for sujeto in sorted(sujetos):
        # Filtrar datos para este sujeto
        df_sujeto = df[df['Sujeto'] == sujeto].copy()
        
        # Ordenar por categoría y muestra
        df_sujeto.sort_values(by=['Categoría', 'Muestra'], inplace=True)
        
        # Crear hoja
        ws = wb.create_sheet(title=sujeto)
        
        # Encabezados
        headers = ['Categoría', 'Muestra', 'Ruta del Video', 'Frames Original', 'Keyframes']
        for col_idx, header in enumerate(headers, 1):
            cell = ws.cell(row=1, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Agregar datos
        row_idx = 2
        current_category = None
        category_start_row = row_idx
        total_frames_category = 0
        total_keyframes_category = 0
        total_muestras_category = 0
        
        total_frames_sujeto = 0
        total_keyframes_sujeto = 0
        total_muestras_sujeto = 0
        total_categories_sujeto = 0
        
        categorias_procesadas = set()
        
        for _, row in df_sujeto.iterrows():
            # Si cambia la categoría, agregar fila de total para categoría anterior
            if current_category is not None and current_category != row['Categoría']:
                # Agregar total de la categoría anterior
                ws.cell(row=row_idx, column=1, value=f"Total {current_category}")
                ws.cell(row=row_idx, column=1).font = total_font
                ws.cell(row=row_idx, column=1).fill = total_fill
                
                ws.cell(row=row_idx, column=2, value=f"{total_muestras_category} muestras")
                ws.cell(row=row_idx, column=2).font = total_font
                ws.cell(row=row_idx, column=2).fill = total_fill
                
                ws.cell(row=row_idx, column=4, value=total_frames_category)
                ws.cell(row=row_idx, column=4).font = total_font
                ws.cell(row=row_idx, column=4).fill = total_fill
                
                ws.cell(row=row_idx, column=5, value=total_keyframes_category)
                ws.cell(row=row_idx, column=5).font = total_font
                ws.cell(row=row_idx, column=5).fill = total_fill
                
                # Aplicar bordes
                for col_idx in range(1, 6):
                    ws.cell(row=row_idx, column=col_idx).border = border
                
                row_idx += 1
                category_start_row = row_idx
                total_frames_category = 0
                total_keyframes_category = 0
                total_muestras_category = 0
            
            # Establecer categoría actual
            current_category = row['Categoría']
            if current_category not in categorias_procesadas:
                categorias_procesadas.add(current_category)
                total_categories_sujeto += 1
            
            # Agregar datos de la fila
            ws.cell(row=row_idx, column=1, value=row['Categoría'])
            ws.cell(row=row_idx, column=2, value=row['Muestra'])
            
            # Agregar ruta del video (o de RGB si no hay video)
            if row['Ruta_Video']:
                ws.cell(row=row_idx, column=3, value=row['Ruta_Video'])
                # Crear hipervínculo
                ws.cell(row=row_idx, column=3).hyperlink = f"file:///{row['Ruta_Video']}"
                ws.cell(row=row_idx, column=3).style = "Hyperlink"
            elif row['Ruta_RGB']:
                ws.cell(row=row_idx, column=3, value=row['Ruta_RGB'])
                # Crear hipervínculo
                ws.cell(row=row_idx, column=3).hyperlink = f"file:///{row['Ruta_RGB']}"
                ws.cell(row=row_idx, column=3).style = "Hyperlink"
            
            ws.cell(row=row_idx, column=4, value=row['Frames_Original'])
            ws.cell(row=row_idx, column=5, value=row['Keyframes'])
            
            # Aplicar bordes
            for col_idx in range(1, 6):
                ws.cell(row=row_idx, column=col_idx).border = border
            
            # Acumular totales
            total_frames_category += row['Frames_Original']
            total_keyframes_category += row['Keyframes']
            total_muestras_category += 1
            
            total_frames_sujeto += row['Frames_Original']
            total_keyframes_sujeto += row['Keyframes']
            total_muestras_sujeto += 1
            
            row_idx += 1
        
        # Agregar total para la última categoría
        if current_category is not None:
            ws.cell(row=row_idx, column=1, value=f"Total {current_category}")
            ws.cell(row=row_idx, column=1).font = total_font
            ws.cell(row=row_idx, column=1).fill = total_fill
            
            ws.cell(row=row_idx, column=2, value=f"{total_muestras_category} muestras")
            ws.cell(row=row_idx, column=2).font = total_font
            ws.cell(row=row_idx, column=2).fill = total_fill
            
            ws.cell(row=row_idx, column=4, value=total_frames_category)
            ws.cell(row=row_idx, column=4).font = total_font
            ws.cell(row=row_idx, column=4).fill = total_fill
            
            ws.cell(row=row_idx, column=5, value=total_keyframes_category)
            ws.cell(row=row_idx, column=5).font = total_font
            ws.cell(row=row_idx, column=5).fill = total_fill
            
            # Aplicar bordes
            for col_idx in range(1, 6):
                ws.cell(row=row_idx, column=col_idx).border = border
            
            row_idx += 1
        
        # Agregar total del sujeto
        ws.cell(row=row_idx, column=1, value=f"TOTAL {sujeto}")
        ws.cell(row=row_idx, column=1).font = Font(bold=True, size=12)
        ws.cell(row=row_idx, column=1).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
        
        ws.cell(row=row_idx, column=2, value=f"{total_muestras_sujeto} muestras")
        ws.cell(row=row_idx, column=2).font = Font(bold=True, size=12)
        ws.cell(row=row_idx, column=2).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
        
        ws.cell(row=row_idx, column=4, value=total_frames_sujeto)
        ws.cell(row=row_idx, column=4).font = Font(bold=True, size=12)
        ws.cell(row=row_idx, column=4).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
        
        ws.cell(row=row_idx, column=5, value=total_keyframes_sujeto)
        ws.cell(row=row_idx, column=5).font = Font(bold=True, size=12)
        ws.cell(row=row_idx, column=5).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
        
        # Aplicar bordes
        for col_idx in range(1, 6):
            ws.cell(row=row_idx, column=col_idx).border = border
        
        # Guardar datos para el resumen
        resumen_stats[sujeto] = {
            'categorias': total_categories_sujeto,
            'muestras': total_muestras_sujeto,
            'frames': total_frames_sujeto,
            'keyframes': total_keyframes_sujeto,
            'promedio_keyframes': total_keyframes_sujeto / total_muestras_sujeto if total_muestras_sujeto > 0 else 0
        }
        
        # Ajustar ancho de columnas
        ws.column_dimensions[get_column_letter(1)].width = 20  # Categoría
        ws.column_dimensions[get_column_letter(2)].width = 15  # Muestra
        ws.column_dimensions[get_column_letter(3)].width = 80  # Ruta
        ws.column_dimensions[get_column_letter(4)].width = 15  # Frames
        ws.column_dimensions[get_column_letter(5)].width = 15  # Keyframes
        
        # Aplicar autofilter
        ws.auto_filter.ref = f"A1:E{row_idx}"
    
    # Crear hoja de resumen
    ws_resumen = wb.create_sheet(title="Resumen", index=0)
    
    # Encabezados del resumen
    headers_resumen = ['Sujeto', 'Total Categorías', 'Total Muestras', 'Total Frames Original', 
                       'Total Keyframes', 'Promedio Keyframes/Muestra']
    
    for col_idx, header in enumerate(headers_resumen, 1):
        cell = ws_resumen.cell(row=1, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.border = border
        cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Agregar datos al resumen
    row_idx = 2
    total_categorias = 0
    total_muestras = 0
    total_frames = 0
    total_keyframes = 0
    
    for sujeto, stats in sorted(resumen_stats.items()):
        ws_resumen.cell(row=row_idx, column=1, value=sujeto)
        ws_resumen.cell(row=row_idx, column=2, value=stats['categorias'])
        ws_resumen.cell(row=row_idx, column=3, value=stats['muestras'])
        ws_resumen.cell(row=row_idx, column=4, value=stats['frames'])
        ws_resumen.cell(row=row_idx, column=5, value=stats['keyframes'])
        ws_resumen.cell(row=row_idx, column=6, value=round(stats['promedio_keyframes'], 1))
        
        # Aplicar bordes
        for col_idx in range(1, 7):
            ws_resumen.cell(row=row_idx, column=col_idx).border = border
        
        # Acumular totales
        total_categorias += stats['categorias']
        total_muestras += stats['muestras']
        total_frames += stats['frames']
        total_keyframes += stats['keyframes']
        
        row_idx += 1
    
    # Agregar fila de totales al resumen
    ws_resumen.cell(row=row_idx, column=1, value="TOTAL")
    ws_resumen.cell(row=row_idx, column=1).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=1).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    ws_resumen.cell(row=row_idx, column=2, value=total_categorias)
    ws_resumen.cell(row=row_idx, column=2).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=2).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    ws_resumen.cell(row=row_idx, column=3, value=total_muestras)
    ws_resumen.cell(row=row_idx, column=3).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=3).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    ws_resumen.cell(row=row_idx, column=4, value=total_frames)
    ws_resumen.cell(row=row_idx, column=4).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=4).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    ws_resumen.cell(row=row_idx, column=5, value=total_keyframes)
    ws_resumen.cell(row=row_idx, column=5).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=5).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    promedio_global = total_keyframes / total_muestras if total_muestras > 0 else 0
    ws_resumen.cell(row=row_idx, column=6, value=round(promedio_global, 1))
    ws_resumen.cell(row=row_idx, column=6).font = Font(bold=True, size=12)
    ws_resumen.cell(row=row_idx, column=6).fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")
    
    # Aplicar bordes
    for col_idx in range(1, 7):
        ws_resumen.cell(row=row_idx, column=col_idx).border = border
    
    # Ajustar ancho de columnas en resumen
    ws_resumen.column_dimensions[get_column_letter(1)].width = 15  # Sujeto
    ws_resumen.column_dimensions[get_column_letter(2)].width = 15  # Categorías
    ws_resumen.column_dimensions[get_column_letter(3)].width = 15  # Muestras
    ws_resumen.column_dimensions[get_column_letter(4)].width = 20  # Frames
    ws_resumen.column_dimensions[get_column_letter(5)].width = 15  # Keyframes
    ws_resumen.column_dimensions[get_column_letter(6)].width = 25  # Promedio
    
    # Guardar el archivo Excel
    wb.save(ruta_excel)
    print(f"Excel guardado en: {ruta_excel}")

# Función principal
def main():
    # Configuración
    ruta_base_datos = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data"
    ruta_excel = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\inventario_videos.xlsx"
    
    print("===== GENERADOR DE INVENTARIO DE VIDEOS =====")
    print(f"Analizando datos en: {ruta_base_datos}")
    
    # Recopilar datos
    df = encontrar_videos_y_keyframes(ruta_base_datos)
    
    if df is not None and not df.empty:
        # Crear Excel
        crear_excel_con_hojas_por_sujeto(df, ruta_excel)
        print(f"\n✅ Proceso completado. Se han encontrado {len(df)} videos/muestras.")
        print(f"El archivo Excel se ha guardado en: {ruta_excel}")
    else:
        print("❌ No se encontraron datos para crear el Excel.")

if __name__ == "__main__":
    main()

===== GENERADOR DE INVENTARIO DE VIDEOS =====
Analizando datos en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data
Encontrados 8 sujetos
Procesando SUJETO 11...
  Procesando expresión: A ver
    Procesando muestra_1
      Encontrados 52 frames en carpeta RGB
    Procesando muestra_2
      Encontrados 54 frames en carpeta RGB
    Procesando muestra_3
      Encontrados 50 frames en carpeta RGB
    Procesando muestra_4
      Encontrados 53 frames en carpeta RGB
    Procesando muestra_5
      Encontrados 44 frames en carpeta RGB
  Procesando expresión: Aburrido
    Procesando muestra_1
      Encontrados 40 frames en carpeta RGB
    Procesando muestra_2
      Encontrados 44 frames en carpeta RGB
    Procesando muestra_3
      Encontrados 65 frames en carpeta RGB
    Procesando muestra_4
      Encontrados 43 frames en carpeta RGB
    Procesando muestra_5
      Encontrados 51 frames en carpeta RGB
  Procesando expresión: Cansado
    Procesando muestra_1
      Encont